<a href="https://colab.research.google.com/github/kittikhun62/efficientnet_keras_transfer_learning/blob/master/regression_test_2_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##เรียกใช้ CSV

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
import numpy as np

In [24]:
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
 !git clone https://github.com/Wanita-8943/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/

/content
/content/efficientnet_keras_transfer_learning


In [25]:
import sys
sys.path.append('/content/drive/My Drive/new/2Class_regression_freeze_1000.h5')

from efficientnet.layers import Swish, DropConnect
from efficientnet.model import ConvKernalInitializer
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({
    'ConvKernalInitializer': ConvKernalInitializer,
    'Swish': Swish,
    'DropConnect':DropConnect
})

In [26]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/new/2Class_regression_freeze_1000.h5')
height = width = model.input_shape[1]

In [27]:
df = pd.read_csv (f'/content/drive/My Drive/data - 2 class Regress.csv')
df

,No,Name_file,Name_Paper,journal,path_Picture,detail,Class,BET,Size(mico),Class_01
0,1,pore-sb,Preparation and electrochemical behaviour of b...,Korean J. Chem. Eng,/content/drive/My Drive/new train/pore-sb/PCC(...,zoom1,0-800,135.06,10,0
1,2,pore-sb,Preparation and electrochemical behaviour of b...,Korean J. Chem. Eng,/content/drive/My Drive/new train/pore-sb/PCC(...,zoom2,0-800,135.06,10,0
2,3,pore-sb,Preparation and electrochemical behaviour of b...,Korean J. Chem. Eng,/content/drive/My Drive/new train/pore-sb/PCC(...,zoom3,0-800,135.06,10,0
3,4,pore-sb,Preparation and electrochemical behaviour of b...,Korean J. Chem. Eng,/content/drive/My Drive/new train/pore-sb/PCC(...,zoom4,0-800,135.06,10,0
4,5,pore-sb,Preparation and electrochemical behaviour of b...,Korean J. Chem. Eng,/content/drive/My Drive/new train/pore-sb/PCC(...,zoom5,0-800,135.06,10,0
...,...,...,...,...,...,...,...,...,...,...
795,796,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom21,0-800,301.70,10,0
796,797,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom22,0-800,301.70,10,0
797,798,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom23,0-800,301.70,10,0
798,799,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom24,0-800,301.70,10,0


In [28]:
train = df[df['No'].between(1,598)]
test = df[df['No'].between(700,800)] 

In [29]:
DATA_PATH = "/content/drive/My Drive/new Regress"
os.chdir(DATA_PATH)
train_dir = os.path.join(DATA_PATH, 'train')
print(train_dir)
test_dir = os.path.join(DATA_PATH, 'test')
print(test_dir)

/content/drive/My Drive/new Regress/train
/content/drive/My Drive/new Regress/test


In [30]:
batch_size = 16

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255, #โมเดลส่วนใหญ่ต้องใช้ RGB ในช่วง 0–1
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe = train,
        directory = train_dir,
        x_col = 'path_Picture',
        y_col = 'Class_01',
        # รูปภาพทั้งหมดจะถูกปรับขนาดตามความสูงและความกว้างของเป้าหมาย
        target_size=(height, width),
        batch_size=batch_size,
        # Since we use categorical_crossentropy loss, we need categorical labels
        #เนื่องจากเราใช้ categorical_crossentropy loss เราจึงต้องมีป้ายกำกับตามหมวดหมู่
        class_mode = 'other')

#label
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# print(labels)

Found 598 validated image filenames.


In [31]:
test = df[df['No'].between(700,800)]
test

,No,Name_file,Name_Paper,journal,path_Picture,detail,Class,BET,Size(mico),Class_01
699,700,c9ra05225a,Study of the pore structure and size effects o...,ROYAL SOCIETY OF CHEMISTRY,/content/drive/My Drive/new train/c9ra05225a/m...,original,801-3200,1980.0,10,1
700,701,jp207232f,Pore Size Effect of Carbon Electrodes on the E...,"Renjie Chen, Zhouying He, Li Li, Feng Wu, Bin ...",/content/drive/My Drive/new train/jp207232f/ME...,zoom1,801-3200,1467.0,10,1
701,702,jp207232f,Pore Size Effect of Carbon Electrodes on the E...,"Renjie Chen, Zhouying He, Li Li, Feng Wu, Bin ...",/content/drive/My Drive/new train/jp207232f/ME...,zoom2,801-3200,1467.0,10,1
702,703,jp207232f,Pore Size Effect of Carbon Electrodes on the E...,"Renjie Chen, Zhouying He, Li Li, Feng Wu, Bin ...",/content/drive/My Drive/new train/jp207232f/ME...,zoom3,801-3200,1467.0,10,1
703,704,jp207232f,Pore Size Effect of Carbon Electrodes on the E...,"Renjie Chen, Zhouying He, Li Li, Feng Wu, Bin ...",/content/drive/My Drive/new train/jp207232f/ME...,zoom4,801-3200,1467.0,10,1
...,...,...,...,...,...,...,...,...,...,...
795,796,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom21,0-800,301.7,10,0
796,797,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom22,0-800,301.7,10,0
797,798,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom23,0-800,301.7,10,0
798,799,1-s2.0-S2095268622000210-main,"Integration of preparation of K, Na-embedded a...","Dingzheng Wang,Deqing Zhu,Jian Pan, Zhengqi Gu...",/content/drive/My Drive/new train/1-s2.0-S2095...,zoom24,0-800,301.7,10,0


In [47]:
act = test['BET'].tolist() #ค่าจริง
path = test['path_Picture'].tolist() #path

In [48]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width)) #รูปทุกรูปมีขนาดไม่เท่ากันจึงตั้งไว้ว่าเวลาจะให้ model ดึงรูปมาทำนายให้ดึงรูปเข้ามาตามขนาดที่ตั้งไว้ตามพารามิตเตอร์
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)  #model ไม่สามารถทำนายรูปภาพโดยตรงได้ จึงเเปลงรูปภาพให้เป็น array เเล้วให้โมเดลทำนาย
    # Reshape
    x = x.reshape((1,) + x.shape) # เพิ่ม ไดเมนชั่นของโมเดล ให้เป็น 4 ได้ เมนชั่น โดยที่ 1, คือบอกให้โมเดลนำเข้าทีละ 1 รูปเเล้วค่อยทำนาย เเละ + ไดเมนชั่น คือ 150 ,150 ,3(สีของ RGB)
    x /= 255.
    result = model.predict([x])
    return result[0]

In [49]:
predict_image(path[0])

1/1 [==============================] - 0s 87ms/step


array([994.57007], dtype=float32)

In [50]:
#Predict
pred_list = list()
prob_list = list()
img_path= path
for i in range(0,len(img_path)):
    predict = predict_image(img_path[i])
    result = predict[0]
    pred_list.append(result)
    # prob_list.append(predict[result])

1/1 [==============================] - 0s 30ms/step


In [63]:
predict

array([994.57007], dtype=float32)

In [62]:
result = predict[0]
result

994.57007

In [51]:
pred_list

[994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994.57007,
 994

In [52]:
print(type(pred_list))

<class 'list'>


In [53]:
pred_int = np.around(pred_list)

In [60]:
pred = np.array(pred_int) #แปลงเป็น array
act = np.array(act)

In [55]:
pred

array([995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995., 995., 995., 995., 995., 995., 995., 995., 995., 995.,
       995., 995.], dtype=float32)

In [58]:
import numpy as np
from sklearn.metrics import confusion_matrix

# act = test['BET'].array
# pred = test['path_Picture'].array

cmat = confusion_matrix(act, pred)
print('classifier accuracy = {}%'.format((100.*np.trace(cmat))/(np.sum(cmat))))

#Marking the Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(act, pred))#performance

ValueError: ignored

In [ ]:
#create CF 
data = {'Actual': act,'Predicted' : pred}
df = pd.DataFrame(data, columns=['Actual','Predicted'])
conf_mat = pd.crosstab(df['Actual'],df['Predicted'],rownames=['Actual'],colnames=['Predicted'])

#Confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(act, pred)

#plot Confusion matrix
import seaborn as sns
sns.set()
fig, ax = plt.subplots(figsize=(10, 10))

ax = sns.heatmap(conf_mat, annot=True, fmt="d", cmap="YlGnBu") #Blues,Oranges,Reds
ax.set_title('Confusion matrix',fontsize=20)
ax.set_ylabel('True label',fontsize=18)
ax.set_xlabel('Predicted label',fontsize=18)